#### The code below in Markdown is meant to be run only once

import boto3
import os

s3 = boto3.client('s3', 
            aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
            aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
            aws_session_token="",
            )

bucket = "zrive-ds-data"
parquets = ["orders.parquet", "regulars.parquet", "abandoned_carts.parquet", "inventory.parquet", "users.parquet"]

for parquet in parquets:
    path = f"groceries/sampled-datasets/{parquet}"
    local = f"parquets/{parquet}"
    s3.download_file(bucket, path, local)


In [43]:
import pandas as pd
import plotly.express as px

In [44]:
orders = pd.read_parquet(
    "/home/santheman/data/parquets/orders.parquet", engine="pyarrow"
)
abandoned_carts = pd.read_parquet(
    "/home/santheman/data/parquets/abandoned_carts.parquet", engine="pyarrow"
)
inventory = pd.read_parquet(
    "/home/santheman/data/parquets/inventory.parquet", engine="pyarrow"
)
regulars = pd.read_parquet(
    "/home/santheman/data/parquets/regulars.parquet", engine="pyarrow"
)
users = pd.read_parquet("/home/santheman/data/parquets/users.parquet", engine="pyarrow")

#### Orders

In [45]:
# Quick look at the data
print("Orders:")
display(orders.head())
print("\n")
print("Columns:", orders.columns)
print("\n")
print("Shape:", orders.shape)
print("\n")
print("Info:")
display(orders.info())
print("\n")
print("Data types:")
display(orders.dtypes)
print("\n")
print("Missing values:")
display(orders.isnull().sum())

Orders:


,id,user_id,created_at,order_date,user_order_seq,ordered_items
10,2204073066628,62e271062eb827e411bd73941178d29b022f5f2de9d37f...,2020-04-30 14:32:19,2020-04-30,1,"[33618849693828, 33618860179588, 3361887404045..."
20,2204707520644,bf591c887c46d5d3513142b6a855dd7ffb9cc00697f6f5...,2020-04-30 17:39:00,2020-04-30,1,"[33618835243140, 33618835964036, 3361886244058..."
21,2204838822020,329f08c66abb51f8c0b8a9526670da2d94c0c6eef06700...,2020-04-30 18:12:30,2020-04-30,1,"[33618891145348, 33618893570180, 3361889766618..."
34,2208967852164,f6451fce7b1c58d0effbe37fcb4e67b718193562766470...,2020-05-01 19:44:11,2020-05-01,1,"[33618830196868, 33618846580868, 3361891234624..."
49,2215889436804,68e872ff888303bff58ec56a3a986f77ddebdbe5c279e7...,2020-05-03 21:56:14,2020-05-03,1,"[33667166699652, 33667166699652, 3366717122163..."




Columns: Index(['id', 'user_id', 'created_at', 'order_date', 'user_order_seq',
       'ordered_items'],
      dtype='object')


Shape: (8773, 6)


Info:
<class 'pandas.core.frame.DataFrame'>
Index: 8773 entries, 10 to 64538
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8773 non-null   int64         
 1   user_id         8773 non-null   object        
 2   created_at      8773 non-null   datetime64[us]
 3   order_date      8773 non-null   datetime64[us]
 4   user_order_seq  8773 non-null   int64         
 5   ordered_items   8773 non-null   object        
dtypes: datetime64[us](2), int64(2), object(2)
memory usage: 479.8+ KB


None



Data types:


id                         int64
user_id                   object
created_at        datetime64[us]
order_date        datetime64[us]
user_order_seq             int64
ordered_items             object
dtype: object



Missing values:


id                0
user_id           0
created_at        0
order_date        0
user_order_seq    0
ordered_items     0
dtype: int64

From the view above, we can see that Orders tells when an order was created, when it was placed and what items where ordered. There aren't any missing values (at first glance) and the total of orders is 8773 (without checking for duplicates).

In [46]:
# All ids are unique, meaning that they are not any duplicates.
len(orders["id"].unique()) == len(orders)

True

In [47]:
val_counts_orders = orders["order_date"].value_counts().reset_index()
val_counts_orders = val_counts_orders.sort_values("order_date")
px.histogram(val_counts_orders, x="order_date", y="count")

/home/santheman/.pyenv/versions/zrive-project/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



The number of orders has been growing in the months we have data from. March has only available data until the 14th, which makes sense since given its low sum of values.

An additional analysis could be made to check whether the number of orders is bigger in the second part of each month in relation to the first part. This would help indicate if March will remain below the previous months.

In [48]:
items = orders.ordered_items.copy()

In [49]:
items_explode = items.explode().reset_index().drop(columns="index")
items_explode.value_counts().head(10)

ordered_items 
34081589887108    4487
39284117930116    2658
34137590366340    1459
34081331970180    1170
34284951863428    1133
34284950356100     954
34370361229444     939
33826465153156     884
34284949766276     838
33667268083844     683
Name: count, dtype: int64

Finally, we observe that is clearly one product much above the rest in terms of number of orders: 34081589887108.

Product number 39284117930116 also has a large presence in orders.

#### Abandoned carts

In [50]:
# Quick look at the data
print("Abandoned carts:")
display(abandoned_carts.head())
print("\n")
print("Columns:", abandoned_carts.columns)
print("\n")
print("Shape:", abandoned_carts.shape)
print("\n")
print("Info:")
display(abandoned_carts.info())
print("\n")
print("Data types:")
display(abandoned_carts.dtypes)
print("\n")
print("Missing values:")
display(abandoned_carts.isnull().sum())

Abandoned carts:


,id,user_id,created_at,variant_id
0,12858560217220,5c4e5953f13ddc3bc9659a3453356155e5efe4739d7a2b...,2020-05-20 13:53:24,"[33826459287684, 33826457616516, 3366719212762..."
13,20352449839236,9d6187545c005d39e44d0456d87790db18611d7c7379bd...,2021-06-27 05:24:13,"[34415988179076, 34037940158596, 3450282236326..."
45,20478401413252,e83fb0273d70c37a2968fee107113698fd4f389c442c0b...,2021-07-18 08:23:49,"[34543001337988, 34037939372164, 3411360609088..."
50,20481783103620,10c42e10e530284b7c7c50f3a23a98726d5747b8128084...,2021-07-18 21:29:36,"[33667268116612, 34037940224132, 3443605520397..."
52,20485321687172,d9989439524b3f6fc4f41686d043f315fb408b954d6153...,2021-07-19 12:17:05,"[33667268083844, 34284950454404, 33973246886020]"




Columns: Index(['id', 'user_id', 'created_at', 'variant_id'], dtype='object')


Shape: (5457, 4)


Info:
<class 'pandas.core.frame.DataFrame'>
Index: 5457 entries, 0 to 70050
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          5457 non-null   int64         
 1   user_id     5457 non-null   object        
 2   created_at  5457 non-null   datetime64[us]
 3   variant_id  5457 non-null   object        
dtypes: datetime64[us](1), int64(1), object(2)
memory usage: 213.2+ KB


None



Data types:


id                     int64
user_id               object
created_at    datetime64[us]
variant_id            object
dtype: object



Missing values:


id            0
user_id       0
created_at    0
variant_id    0
dtype: int64

The data shows all discarded carts that were initiated, by whom and when they were abandoned and what products they had.

There are no missing values. At first glance, all data looks good.

In [51]:
val_counts = abandoned_carts["created_at"].value_counts().reset_index()
val_counts = val_counts.sort_values("created_at")
px.histogram(val_counts, x="created_at", y="count")

/home/santheman/.pyenv/versions/zrive-project/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [52]:
val_counts.head(2)

,created_at,count
0,2020-05-20 13:53:24,1
2729,2021-06-27 05:24:13,1


Same as before, an analysis is performed on the dates when the most carts were abandoned, this time in weeks. It is noticeable how only from 2021-06-27 we start getting abandoned carts.

It may acceptable to consider the first instance a test for the application from which this data is from, but we cannot be sure. Maybe ask the provider (?)

Also, notice a big peak in January 2022.

In [53]:
# Top 5 users with the most abandoned carts
val_counts_ac = abandoned_carts["user_id"].value_counts().reset_index()
val_counts_ac.head(5)

,user_id,count
0,257be7ae940425880bbb20bf162c2616b32881bf0a8bda...,10
1,1dacfd2a360677052d8605f843ae410dd23b0ddb7f506c...,9
2,fffd9f989509e36d1fc3e3e53627d6341482f385052a03...,7
3,a8ea4d1ff9cfc5005b7354d1d17564347dd842bab2a6c3...,7
4,97e81469f5758878f4d7eaa3af6b4fc37b2b5c22558811...,7


#### Inventory

In [54]:
# Quick look at the data
print("Inventory:")
display(inventory.head())
print("\n")
print("Columns:", inventory.columns)
print("\n")
print("Shape:", inventory.shape)
print("\n")
print("Info:")
display(inventory.info())
print("\n")
print("Data types:")
display(inventory.dtypes)
print("\n")
print("Missing values:")
display(inventory.isnull().sum())

Inventory:


,variant_id,price,compare_at_price,vendor,product_type,tags
0,39587297165444,3.09,3.15,heinz,condiments-dressings,"[table-sauces, vegan]"
1,34370361229444,4.99,5.50,whogivesacrap,toilet-roll-kitchen-roll-tissue,"[b-corp, eco, toilet-rolls]"
2,34284951863428,3.69,3.99,plenty,toilet-roll-kitchen-roll-tissue,[kitchen-roll]
3,33667283583108,1.79,1.99,thecheekypanda,toilet-roll-kitchen-roll-tissue,"[b-corp, cruelty-free, eco, tissue, vegan]"
4,33803537973380,1.99,2.09,colgate,dental,[dental-accessories]




Columns: Index(['variant_id', 'price', 'compare_at_price', 'vendor', 'product_type',
       'tags'],
      dtype='object')


Shape: (1733, 6)


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   variant_id        1733 non-null   int64  
 1   price             1733 non-null   float64
 2   compare_at_price  1733 non-null   float64
 3   vendor            1733 non-null   object 
 4   product_type      1733 non-null   object 
 5   tags              1733 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 81.4+ KB


None



Data types:


variant_id            int64
price               float64
compare_at_price    float64
vendor               object
product_type         object
tags                 object
dtype: object



Missing values:


variant_id          0
price               0
compare_at_price    0
vendor              0
product_type        0
tags                0
dtype: int64

This dataset describes all products in inventory good for sale, with adjacent data such as price, vendor or others. There aren't any missing values. It would be interesting to see which products are among the most bought among orders

In [55]:
items_val_counts = items_explode.value_counts().reset_index()
top10_list = items_val_counts['ordered_items'].head(10).to_list()
top10_df = items_val_counts.head(10).merge(inventory, left_on="ordered_items", right_on="variant_id", how="left").drop(columns="variant_id")
top10_df

,ordered_items,count,price,compare_at_price,vendor,product_type,tags
0,34081589887108,4487,10.79,11.94,oatly,long-life-milk-substitutes,"[oat-milk, vegan]"
1,39284117930116,2658,NaN,NaN,NaN,NaN,NaN
2,34137590366340,1459,NaN,NaN,NaN,NaN,NaN
3,34081331970180,1170,NaN,NaN,NaN,NaN,NaN
4,34284951863428,1133,3.69,3.99,plenty,toilet-roll-kitchen-roll-tissue,[kitchen-roll]
5,34284950356100,954,1.99,3.00,fairy,dishwashing,"[discontinue, swapped, washing-up-liquid]"
6,34370361229444,939,4.99,5.50,whogivesacrap,toilet-roll-kitchen-roll-tissue,"[b-corp, eco, toilet-rolls]"
7,33826465153156,884,1.89,1.99,clearspring,tins-packaged-foods,"[gluten-free, meat-alternatives, vegan]"
8,34284949766276,838,8.49,9.00,andrex,toilet-roll-kitchen-roll-tissue,[toilet-rolls]
9,33667268083844,683,15.99,19.99,persil,washing-powder,[washing-powder]


There are missing items in the inventory. Maybe no longer on sale? The top sold is a vegan milk from Oatly. Next in line are kitchen paper rolls, from Plenty.

In [56]:
inventory[inventory.price == 0]

,variant_id,price,compare_at_price,vendor,product_type,tags
95,40070658490500,0.0,0.0,jordans,cereal,[cereal-bars]
96,40167931674756,0.0,0.0,whogivesacrap,,[]
97,40167931707524,0.0,0.0,whogivesacrap,,[]
99,40070656786564,0.0,0.0,mcvities,biscuits-crackers,"[biscuits, pm]"
100,40070657933444,0.0,0.0,astonish,cleaning-products,[bathroom-limescale-cleaner]
...,...,...,...,...,...,...
1554,40070660292740,0.0,0.0,biona,biscuits-crackers,[crackers]
1575,40070658654340,0.0,0.0,lovecorn,snacks-confectionery,[savoury-snacks]
1623,40070660587652,0.0,0.0,snackajack,biscuits-crackers,[crackers]
1627,40070660489348,0.0,0.0,garnier,skincare,[facial-skincare]


Why are there many items that their price is zero? Also, with this slicingm, notice that there are empty tags and product types. Let's look into that.

In [57]:
inventory[inventory.product_type == ""]

,variant_id,price,compare_at_price,vendor,product_type,tags
96,40167931674756,0.00,0.00,whogivesacrap,,[]
97,40167931707524,0.00,0.00,whogivesacrap,,[]
586,40099114188932,0.01,0.00,bother,,"[discontinue, trade-swap]"
1728,39539419218052,4.99,5.99,bother,,[household-sundries]


So not many empty product types and two of them match with empty tags and prices near zero, except for the last instance.

In [58]:
# Trying to get empty tags
inventory[inventory.tags.apply(len) == 0]

,variant_id,price,compare_at_price,vendor,product_type,tags
96,40167931674756,0.0,0.0,whogivesacrap,,[]
97,40167931707524,0.0,0.0,whogivesacrap,,[]


Again, the only empty tags are the same ones that appear with the product_type

In [84]:
#  Quick code with function to calculate how much order cost
cost_of_orders = orders.copy()
cost_map = dict(zip(inventory.variant_id, inventory.price))

# Should look into how to manage the case where an item is not in the inventory, for now the sum is 0, deeming it as not valid
# This is done because I am assuming that for the analysis, outdated inventory items will not be important. Obviously, this is a temporary solution
def f_cost_of_orders(items: list, cost_map: dict) -> float:
    try:
        return sum([cost_map[x] for x in items])
    except KeyError:
        return 0
cost_of_orders['price'] = cost_of_orders['ordered_items'].apply(lambda x: f_cost_of_orders(x, cost_map))

In [90]:
cost_of_orders[['price']].describe().drop('count')

,price
mean,13.940784
std,27.912096
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,319.800000


Analyzing prices we can see that there are way too many zeros for them to be ignored. This is definitely something to look into. Let's make a quick fix, just ignoring elements that are not in the inventory

In [102]:
cost_of_orders = orders.copy()
def f_cost_of_orders_quick_fix(items: list, cost_map: dict) -> float:
    return sum([cost_map[x] for x in items if x in cost_map.keys()])
cost_of_orders['price'] = cost_of_orders['ordered_items'].apply(lambda x: f_cost_of_orders_quick_fix(x, cost_map))

In [104]:
cost_of_orders[['price']].describe().drop('count')

,price
mean,55.832173
std,24.316070
min,0.000000
25%,44.250000
50%,51.860000
75%,60.590000
max,319.800000


This makes a little more sense. Let's see the prices of orders:

In [114]:
fig = px.bar(x=cost_of_orders['order_date'], y=cost_of_orders['price'])
fig.update_xaxes(title_text='Order Date')
fig.update_yaxes(title_text='Total price for a date')

/home/santheman/.pyenv/versions/zrive-project/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



We can see here the amount of prices by order, along the dates the orders where made at. The peak in January 2022 matches that in the number of orders that we saw before.

#### Regulars

In [59]:
# Quick look at the data
print("Regulars:")
display(regulars.head())
print("\n")
print("Columns:", regulars.columns)
print("\n")
print("Shape:", regulars.shape)
print("\n")
print("Info:")
display(regulars.info())
print("\n")
print("Data types:")
display(regulars.dtypes)
print("\n")
print("Missing values:")
display(regulars.isnull().sum())

Regulars:


,user_id,variant_id,created_at
3,68e872ff888303bff58ec56a3a986f77ddebdbe5c279e7...,33618848088196,2020-04-30 15:07:03
11,aed88fc0b004270a62ff1fe4b94141f6b1db1496dbb0c0...,33667178659972,2020-05-05 23:34:35
18,68e872ff888303bff58ec56a3a986f77ddebdbe5c279e7...,33619009208452,2020-04-30 15:07:03
46,aed88fc0b004270a62ff1fe4b94141f6b1db1496dbb0c0...,33667305373828,2020-05-05 23:34:35
47,4594e99557113d5a1c5b59bf31b8704aafe5c7bd180b32...,33667247341700,2020-05-06 14:42:11




Columns: Index(['user_id', 'variant_id', 'created_at'], dtype='object')


Shape: (18105, 3)


Info:
<class 'pandas.core.frame.DataFrame'>
Index: 18105 entries, 3 to 37720
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     18105 non-null  object        
 1   variant_id  18105 non-null  int64         
 2   created_at  18105 non-null  datetime64[us]
dtypes: datetime64[us](1), int64(1), object(1)
memory usage: 565.8+ KB


None



Data types:


user_id               object
variant_id             int64
created_at    datetime64[us]
dtype: object



Missing values:


user_id       0
variant_id    0
created_at    0
dtype: int64

In [60]:
# Could be interesting to which products from the inventory are the most popular
merged_reg = regulars.merge(inventory, on="variant_id", how="inner")
merged_reg.head()

,user_id,variant_id,created_at,price,compare_at_price,vendor,product_type,tags
0,4594e99557113d5a1c5b59bf31b8704aafe5c7bd180b32...,33667247341700,2020-05-06 14:42:11,3.49,3.5,method,cleaning-products,"[cruelty-free, eco, vegan, window-glass-cleaner]"
1,62d160f3f142d657a5d12f54992cd76a9de1ed6d75cc33...,33667247341700,2021-12-01 08:50:22,3.49,3.5,method,cleaning-products,"[cruelty-free, eco, vegan, window-glass-cleaner]"
2,2832544f79be3f761f54df8a105d6428c99d4ad313ca77...,33667247341700,2022-01-17 22:19:10,3.49,3.5,method,cleaning-products,"[cruelty-free, eco, vegan, window-glass-cleaner]"
3,2083896edc0f9cff38ce3286cf9b917b223a9287a238e5...,33667247341700,2020-05-24 10:41:21,3.49,3.5,method,cleaning-products,"[cruelty-free, eco, vegan, window-glass-cleaner]"
4,94a1bf4d9f5e0c4f282d6c1233e3a0e4738d4e7ecee4bd...,33667247341700,2020-07-03 21:33:45,3.49,3.5,method,cleaning-products,"[cruelty-free, eco, vegan, window-glass-cleaner]"


In [61]:
val_counts_reg = merged_reg['product_type'].value_counts()
px.bar(val_counts_reg).update_layout(title='Product Type count among regulars')

We can see that the most regular product types are those for cleaning products and those for packaging food. The same analysis could be performed to check the most popular vendors among regulars.

#### Users

In [62]:
# Quick look at the data
print("Users:")
display(users.head())
print("\n")
print("Columns:", users.columns)
print("\n")
print("Shape:", users.shape)
print("\n")
print("Info:")
display(users.info())
print("\n")
print("Data types:")
display(users.dtypes)
print("\n")
print("Missing values:")
display(users.isnull().sum())

Users:


,user_id,user_segment,user_nuts1,first_ordered_at,customer_cohort_month,count_people,count_adults,count_children,count_babies,count_pets
2160,0e823a42e107461379e5b5613b7aa00537a72e1b0eaa7a...,Top Up,UKH,2021-05-08 13:33:49,2021-05-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1123,15768ced9bed648f745a7aa566a8895f7a73b9a47c1d4f...,Top Up,UKJ,2021-11-17 16:30:20,2021-11-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1958,33e0cb6eacea0775e34adbaa2c1dec16b9d6484e6b9324...,Top Up,UKD,2022-03-09 23:12:25,2022-03-01 00:00:00,NaN,NaN,NaN,NaN,NaN
675,57ca7591dc79825df0cecc4836a58e6062454555c86c35...,Top Up,UKI,2021-04-23 16:29:02,2021-04-01 00:00:00,NaN,NaN,NaN,NaN,NaN
4694,085d8e598139ce6fc9f75d9de97960fa9e1457b409ec00...,Top Up,UKJ,2021-11-02 13:50:06,2021-11-01 00:00:00,NaN,NaN,NaN,NaN,NaN




Columns: Index(['user_id', 'user_segment', 'user_nuts1', 'first_ordered_at',
       'customer_cohort_month', 'count_people', 'count_adults',
       'count_children', 'count_babies', 'count_pets'],
      dtype='object')


Shape: (4983, 10)


Info:
<class 'pandas.core.frame.DataFrame'>
Index: 4983 entries, 2160 to 3360
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                4983 non-null   object 
 1   user_segment           4983 non-null   object 
 2   user_nuts1             4932 non-null   object 
 3   first_ordered_at       4983 non-null   object 
 4   customer_cohort_month  4983 non-null   object 
 5   count_people           325 non-null    float64
 6   count_adults           325 non-null    float64
 7   count_children         325 non-null    float64
 8   count_babies           325 non-null    float64
 9   count_pets             325 non-null    float64
dtypes: float64(5), ob

None



Data types:


user_id                   object
user_segment              object
user_nuts1                object
first_ordered_at          object
customer_cohort_month     object
count_people             float64
count_adults             float64
count_children           float64
count_babies             float64
count_pets               float64
dtype: object



Missing values:


user_id                     0
user_segment                0
user_nuts1                 51
first_ordered_at            0
customer_cohort_month       0
count_people             4658
count_adults             4658
count_children           4658
count_babies             4658
count_pets               4658
dtype: int64

In [63]:
users.describe()

,count_people,count_adults,count_children,count_babies,count_pets
count,325.000000,325.000000,325.000000,325.000000,325.000000
mean,2.787692,2.003077,0.707692,0.076923,0.636923
std,1.365753,0.869577,1.026246,0.289086,0.995603
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,0.000000,0.000000,0.000000
50%,3.000000,2.000000,0.000000,0.000000,0.000000
75%,4.000000,2.000000,1.000000,0.000000,1.000000
max,8.000000,7.000000,6.000000,2.000000,6.000000


Users has an important amount of null values for the column that start with count_. Let's try and see what's going on:

In [64]:
users.dropna().head()

,user_id,user_segment,user_nuts1,first_ordered_at,customer_cohort_month,count_people,count_adults,count_children,count_babies,count_pets
4751,09d70e0b0778117aec5550c08032d56f8e06f992741680...,Proposition,UKI,2021-06-28 12:07:04,2021-06-01 00:00:00,1.0,1.0,0.0,0.0,0.0
3154,4f5ff38ce5ed48096ba80dff80e167db1ad24b9ebdb00c...,Top Up,UKD,2020-06-12 12:07:35,2020-06-01 00:00:00,3.0,2.0,0.0,1.0,0.0
736,7b2ae50bb11646436fa613394fc3e71e1a0cdc3ba30cdb...,Proposition,UKF,2020-10-03 09:53:57,2020-10-01 00:00:00,2.0,2.0,0.0,0.0,2.0
4792,5e977a4aa2c57f306b8a22f92eaaa177f7dc31a52df82c...,Proposition,UKI,2021-10-14 10:41:13,2021-10-01 00:00:00,2.0,1.0,1.0,0.0,1.0
2217,eafb89ad33eb377adb98a915b6a5a65f1284c2db517d07...,Proposition,UKH,2022-01-20 15:53:09,2022-01-01 00:00:00,2.0,2.0,0.0,0.0,0.0


The columns count_ may refer to the number of instances of the class that are at the store at the moment. Still, can't figure out why there are some many missing values. Maybe ask the provider (?).

## EDA Feature Frame

In [65]:
route_data = r"/home/santheman/data/csv/feature_frame.csv"
data = pd.read_csv(route_data)

In [66]:
# This is to work on a separate df and avoid messing with the OG data
df_data = data.copy()

In [67]:
print("Data:")
display(df_data.head())
print("\n")
print("Columns:", df_data.columns)
print("\n")
print("Shape:", df_data.shape)
print("\n")
print("Info:")
display(df_data.info())
print("\n")
print("Data types:")
display(df_data.dtypes)
print("\n")
print("Missing values:")
display(df_data.isnull().sum())

Data:


,variant_id,product_type,order_id,user_id,created_at,order_date,user_order_seq,outcome,ordered_before,abandoned_before,...,count_children,count_babies,count_pets,people_ex_baby,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type
0,33826472919172,ricepastapulses,2807985930372,3482464092292,2020-10-05 16:46:19,2020-10-05 00:00:00,3,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
1,33826472919172,ricepastapulses,2808027644036,3466586718340,2020-10-05 17:59:51,2020-10-05 00:00:00,2,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
2,33826472919172,ricepastapulses,2808099078276,3481384026244,2020-10-05 20:08:53,2020-10-05 00:00:00,4,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
3,33826472919172,ricepastapulses,2808393957508,3291363377284,2020-10-06 08:57:59,2020-10-06 00:00:00,2,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
4,33826472919172,ricepastapulses,2808429314180,3537167515780,2020-10-06 10:37:05,2020-10-06 00:00:00,3,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618




Columns: Index(['variant_id', 'product_type', 'order_id', 'user_id', 'created_at',
       'order_date', 'user_order_seq', 'outcome', 'ordered_before',
       'abandoned_before', 'active_snoozed', 'set_as_regular',
       'normalised_price', 'discount_pct', 'vendor', 'global_popularity',
       'count_adults', 'count_children', 'count_babies', 'count_pets',
       'people_ex_baby', 'days_since_purchase_variant_id',
       'avg_days_to_buy_variant_id', 'std_days_to_buy_variant_id',
       'days_since_purchase_product_type', 'avg_days_to_buy_product_type',
       'std_days_to_buy_product_type'],
      dtype='object')


Shape: (2880549, 27)


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880549 entries, 0 to 2880548
Data columns (total 27 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   variant_id                        int64  
 1   product_type                      object 
 2   order_id                          int64

None



Data types:


variant_id                            int64
product_type                         object
order_id                              int64
user_id                               int64
created_at                           object
order_date                           object
user_order_seq                        int64
outcome                             float64
ordered_before                      float64
abandoned_before                    float64
active_snoozed                      float64
set_as_regular                      float64
normalised_price                    float64
discount_pct                        float64
vendor                               object
global_popularity                   float64
count_adults                        float64
count_children                      float64
count_babies                        float64
count_pets                          float64
people_ex_baby                      float64
days_since_purchase_variant_id      float64
avg_days_to_buy_variant_id      



Missing values:


variant_id                          0
product_type                        0
order_id                            0
user_id                             0
created_at                          0
order_date                          0
user_order_seq                      0
outcome                             0
ordered_before                      0
abandoned_before                    0
active_snoozed                      0
set_as_regular                      0
normalised_price                    0
discount_pct                        0
vendor                              0
global_popularity                   0
count_adults                        0
count_children                      0
count_babies                        0
count_pets                          0
people_ex_baby                      0
days_since_purchase_variant_id      0
avg_days_to_buy_variant_id          0
std_days_to_buy_variant_id          0
days_since_purchase_product_type    0
avg_days_to_buy_product_type        0
std_days_to_

1. There aren't any missing data. All columns have a logical sense, regarding it's dtype and its values.
2. Maybe consider changing the dtype to datetime in 'created_at' and 'order_date', rather than a string.
3. There are several flags (0 or 1)

In [68]:
df_data['order_date'] = pd.to_datetime(df_data['order_date'])
df_data['created_at'] = pd.to_datetime(df_data['created_at'])

In [69]:
df_data.columns

Index(['variant_id', 'product_type', 'order_id', 'user_id', 'created_at',
       'order_date', 'user_order_seq', 'outcome', 'ordered_before',
       'abandoned_before', 'active_snoozed', 'set_as_regular',
       'normalised_price', 'discount_pct', 'vendor', 'global_popularity',
       'count_adults', 'count_children', 'count_babies', 'count_pets',
       'people_ex_baby', 'days_since_purchase_variant_id',
       'avg_days_to_buy_variant_id', 'std_days_to_buy_variant_id',
       'days_since_purchase_product_type', 'avg_days_to_buy_product_type',
       'std_days_to_buy_product_type'],
      dtype='object')

In [70]:
# Too many columns to try a .describe() of the whole dataset -> let's analyze columns that seem interesting
df_data[['normalised_price', 'discount_pct', 'global_popularity', 'count_adults', 'count_children', 'count_babies', 'count_pets', 'days_since_purchase_variant_id']].describe().drop('count')

,normalised_price,discount_pct,global_popularity,count_adults,count_children,count_babies,count_pets,days_since_purchase_variant_id
mean,0.127281,0.186274,0.010703,2.017627,0.054922,0.003539,0.051341,33.129610
std,0.126838,0.193448,0.016634,0.209892,0.327659,0.059380,0.301365,3.707162
min,0.015993,-0.040161,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.053944,0.084622,0.001629,2.000000,0.000000,0.000000,0.000000,33.000000
50%,0.081052,0.116918,0.006284,2.000000,0.000000,0.000000,0.000000,33.000000
75%,0.135267,0.223464,0.014184,2.000000,0.000000,0.000000,0.000000,33.000000
max,1.000000,1.325301,0.425439,5.000000,3.000000,1.000000,6.000000,148.000000


In the light of this data we can assume:
1. People take mostly 33 days to buy a product again
2. It is rather not normal to have children, babies or pets. The most common is to be two adults.

In [71]:
# Quick view of the flag's different comnbinations
val_counts = df_data.value_counts(subset=['ordered_before', 'abandoned_before', 'outcome'])
# Ordered before
print("Ordered products before")
display(val_counts.loc[1])
# Not ordered before
print("Didn't order products before")
display(val_counts.loc[0])

Ordered products before


abandoned_before  outcome
0.0               0.0        50563
                  1.0         9716
1.0               1.0          329
                  0.0          283
Name: count, dtype: int64

Didn't order products before


abandoned_before  outcome
0.0               0.0        2796259
                  1.0          22256
1.0               1.0            931
                  0.0            212
Name: count, dtype: int64

In [72]:
# Now let's check orders by user
df_order_counter = df_data.groupby('user_id').agg({'order_id': 'nunique'}).sort_values('order_id', ascending=False)
df_order_counter.value_counts()

order_id
1           1148
2            416
3            201
4             82
5             55
6             13
7             11
8              5
9              2
10             2
11             1
16             1
Name: count, dtype: int64

Insights:
1. Most people have only ever made one or two orders
2. Very few people have only ever made more than seven orders or more

In [73]:
df_product_counter = df_data \
    .groupby('product_type') \
    .agg({'order_id': 'count'}) \
    .rename(columns={'order_id': 'types_count'}) \
    .sort_values(by='types_count', ascending=False)

df_product_counter['% over total'] = df_product_counter['types_count']*100/df_product_counter['types_count'].sum()
display(df_product_counter.head(10))
display(df_product_counter.tail(10).sort_values(by='% over total', ascending=True))

,types_count,% over total
product_type,,
tinspackagedfoods,226474,7.862182
condimentsdressings,129749,4.504315
ricepastapulses,128098,4.447000
haircare,114978,3.991531
cookingingredientsoils,110686,3.842531
longlifemilksubstitutes,110235,3.826875
dishwasherdetergent,98144,3.407128
cereal,90898,3.155579
bathroomlimescalecleaner,82632,2.868620


,types_count,% over total
product_type,,
premixedcocktails,2620,0.090955
feedingweaning,2790,0.096857
petcare,4075,0.141466
householdsundries,6735,0.233810
babyfood12months,6797,0.235962
babymilkformula,7305,0.253597
washingcapsules,7527,0.261304
maternity,8495,0.294909
beer,9796,0.340074


Insights:
1. The most popular products are packaged foods, condiments and dressings, rice and pasta, and haircare
2. The least popular products are premixed cocktails, pet care or baby stuff (weaning, baby milf formula, baby food, maternity) in general. This makes sense, given that we previously analyzed that there were very few buyers with pets and/or babies in the dataset, mostly adults.